In [3]:
# Import required packages
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.noise import GaussianNoise
from keras.layers.normalization import BatchNormalization
from keras.applications import ResNet50, VGG16, InceptionV3
from keras.applications.vgg16 import preprocess_input, decode_predictions
# from utils import make_parallel
import os
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from keras.utils import to_categorical
from tqdm import tqdm
%matplotlib inline

Using TensorFlow backend.


## Method 1: Apply pre-trained model on training data directly

**Here I use VGG16 model.**

**Steps** 

- Load image：cv2.imread
- Modify size：cv2.resize, the input image size of VGG16 is 224 * 224
- Modify dimension：np.expand_dims, the input of VGG16 is 3 channels
- Preprocess：preprocess_input，VGG16 uses mean substraction
- Predict：model.predict
- Decode result：decode_predictions, decode the results into a list of tuples (class, description, probability). E.g. (u'n02504013', u'Indian_elephant', 0.82658225)

In [5]:
model1 = VGG16(weights='imagenet', include_top=True)

def inference(imgs, preprocess=True):
    plt.figure(figsize = (15, 7)) # set figure size
    for i in range(len(imgs)):
        plt.subplot(1, len(imgs), i+1)
        img = cv2.imread(imgs[i])
        img = img[:, :, ::-1]
        x = cv2.resize(img, dsize=(224, 224))
        x = np.expand_dims(x, axis=0) # expand to 3 channels
        if preprocess:
            x = preprocess_input(x)
        y_pred = model1.predict(x)
        _, label, prob = decode_predictions(y_pred, top=1)[0][0] # only pick top 1 prediction
        plt.imshow(img, cmap='gray')
        plt.title('%s: %.2f %%' % (label, prob * 100), size = 15)
    plt.show()
    
inference(['./train/cat1.jpg', './train/cat2.jpg', './train/cat3.jpg', './train/cat4.jpg'], False)
inference(['./train/cat1.jpg', './train/cat2.jpg', './train/cat3.jpg', './train/cat4.jpg'])

 18145280/553467096 [..............................] - ETA: 5636 - ETA: 2850 - ETA: 1978 - ETA: 1497 - ETA: 1407 - ETA: 1146 - ETA: 8569s - ETA: 6887 - ETA: 6762 - ETA: 5292 - ETA: 5226 - ETA: 5276 - ETA: 5209 - ETA: 4976 - ETA: 4942 - ETA: 4689 - ETA: 4675 - ETA: 4399 - ETA: 4375 - ETA: 4226 - ETA: 4208 - ETA: 4021 - ETA: 4012 - ETA: 3902 - ETA: 3894 - ETA: 3877 - ETA: 3755 - ETA: 3682 - ETA: 3658 - ETA: 3648 - ETA: 3826 - ETA: 3605 - ETA: 3611 - ETA: 3594 - ETA: 3608 - ETA: 3718 - ETA: 3622 - ETA: 3633 - ETA: 3678 - ETA: 3703 - ETA: 3735 - ETA: 3937 - ETA: 3963 - ETA: 3893 - ETA: 3925 - ETA: 3917 - ETA: 4047 - ETA: 4052 - ETA: 4120 - ETA: 4170 - ETA: 4207 - ETA: 4274 - ETA: 4312 - ETA: 4366 - ETA: 4422 - ETA: 4456 - ETA: 4438 - ETA: 4505 - ETA: 4551 - ETA: 4607 - ETA: 4622 - ETA: 4605 - ETA: 4646 - ETA: 4704 - ETA: 4738 - ETA: 4717 - ETA: 4765 - ETA: 4797 - ETA: 4758 - ETA: 4833 - ETA: 4849 - ETA: 4919 - ETA: 4966 - ETA: 4975 - ETA: 5011 - ETA: 4999 - ETA: 5006 - ETA: 4974 - ETA: 495

 34332672/553467096 [>.............................] - ETA: 6237 - ETA: 6235 - ETA: 6235 - ETA: 6232 - ETA: 6236 - ETA: 6232 - ETA: 6232 - ETA: 6234 - ETA: 6232 - ETA: 6230 - ETA: 6229 - ETA: 6230 - ETA: 6226 - ETA: 6227 - ETA: 6224 - ETA: 6226 - ETA: 6222 - ETA: 6224 - ETA: 6221 - ETA: 6221 - ETA: 6219 - ETA: 6217 - ETA: 6217 - ETA: 6214 - ETA: 6214 - ETA: 6209 - ETA: 6210 - ETA: 6207 - ETA: 6208 - ETA: 6205 - ETA: 6208 - ETA: 6206 - ETA: 6201 - ETA: 6199 - ETA: 6197 - ETA: 6200 - ETA: 6192 - ETA: 6190 - ETA: 6193 - ETA: 6191 - ETA: 6192 - ETA: 6190 - ETA: 6192 - ETA: 6187 - ETA: 6188 - ETA: 6186 - ETA: 6188 - ETA: 6186 - ETA: 6180 - ETA: 6190 - ETA: 6182 - ETA: 6185 - ETA: 6181 - ETA: 6184 - ETA: 6186 - ETA: 6180 - ETA: 6183 - ETA: 6184 - ETA: 6181 - ETA: 6184 - ETA: 6186 - ETA: 6182 - ETA: 6184 - ETA: 6181 - ETA: 6180 - ETA: 6182 - ETA: 6182 - ETA: 6181 - ETA: 6183 - ETA: 6180 - ETA: 6180 - ETA: 6186 - ETA: 6186 - ETA: 6185 - ETA: 6182 - ETA: 6185 - ETA: 6181 - ETA: 6179 - ETA: 6181

 53600256/553467096 [=>............................] - ETA: 5787 - ETA: 5786 - ETA: 5786 - ETA: 5789 - ETA: 5785 - ETA: 5787 - ETA: 5785 - ETA: 5790 - ETA: 5786 - ETA: 5787 - ETA: 5788 - ETA: 5788 - ETA: 5795 - ETA: 5793 - ETA: 5796 - ETA: 5797 - ETA: 5798 - ETA: 5798 - ETA: 5800 - ETA: 5802 - ETA: 5800 - ETA: 5801 - ETA: 5802 - ETA: 5808 - ETA: 5803 - ETA: 5806 - ETA: 5811 - ETA: 5813 - ETA: 5815 - ETA: 5821 - ETA: 5828 - ETA: 5835 - ETA: 5840 - ETA: 5845 - ETA: 5852 - ETA: 5862 - ETA: 5872 - ETA: 5879 - ETA: 5885 - ETA: 5892 - ETA: 5896 - ETA: 5898 - ETA: 5903 - ETA: 5906 - ETA: 5908 - ETA: 5912 - ETA: 5914 - ETA: 5917 - ETA: 5919 - ETA: 5919 - ETA: 5924 - ETA: 5919 - ETA: 5921 - ETA: 5922 - ETA: 5922 - ETA: 5923 - ETA: 5921 - ETA: 5922 - ETA: 5924 - ETA: 5924 - ETA: 5923 - ETA: 5923 - ETA: 5925 - ETA: 5926 - ETA: 5922 - ETA: 5924 - ETA: 5925 - ETA: 5925 - ETA: 5923 - ETA: 5928 - ETA: 5927 - ETA: 5928 - ETA: 5928 - ETA: 5929 - ETA: 5935 - ETA: 5933 - ETA: 5936 - ETA: 5938 - ETA: 5939

 67682304/553467096 [==>...........................] - ETA: 5250 - ETA: 5249 - ETA: 5250 - ETA: 5251 - ETA: 5251 - ETA: 5250 - ETA: 5251 - ETA: 5252 - ETA: 5251 - ETA: 5252 - ETA: 5251 - ETA: 5251 - ETA: 5255 - ETA: 5253 - ETA: 5253 - ETA: 5252 - ETA: 5253 - ETA: 5254 - ETA: 5255 - ETA: 5255 - ETA: 5255 - ETA: 5255 - ETA: 5256 - ETA: 5256 - ETA: 5255 - ETA: 5256 - ETA: 5256 - ETA: 5256 - ETA: 5256 - ETA: 5256 - ETA: 5256 - ETA: 5256 - ETA: 5256 - ETA: 5257 - ETA: 5257 - ETA: 5257 - ETA: 5258 - ETA: 5259 - ETA: 5257 - ETA: 5258 - ETA: 5261 - ETA: 5261 - ETA: 5262 - ETA: 5263 - ETA: 5264 - ETA: 5266 - ETA: 5266 - ETA: 5267 - ETA: 5270 - ETA: 5275 - ETA: 5275 - ETA: 5279 - ETA: 5283 - ETA: 5287 - ETA: 5286 - ETA: 5293 - ETA: 5300 - ETA: 5304 - ETA: 5310 - ETA: 5318 - ETA: 5325 - ETA: 5331 - ETA: 5335 - ETA: 5336 - ETA: 5340 - ETA: 5344 - ETA: 5347 - ETA: 5350 - ETA: 5352 - ETA: 5355 - ETA: 5356 - ETA: 5359 - ETA: 5360 - ETA: 5361 - ETA: 5362 - ETA: 5363 - ETA: 5364 - ETA: 5365 - ETA: 5364

 84484096/553467096 [===>..........................] - ETA: 6407 - ETA: 6402 - ETA: 6395 - ETA: 6395 - ETA: 6393 - ETA: 6391 - ETA: 6390 - ETA: 6388 - ETA: 6385 - ETA: 6385 - ETA: 6382 - ETA: 6382 - ETA: 6380 - ETA: 6377 - ETA: 6377 - ETA: 6378 - ETA: 6373 - ETA: 6370 - ETA: 6369 - ETA: 6367 - ETA: 6366 - ETA: 6364 - ETA: 6364 - ETA: 6363 - ETA: 6363 - ETA: 6362 - ETA: 6360 - ETA: 6357 - ETA: 6357 - ETA: 6355 - ETA: 6354 - ETA: 6352 - ETA: 6349 - ETA: 6348 - ETA: 6345 - ETA: 6344 - ETA: 6345 - ETA: 6342 - ETA: 6340 - ETA: 6338 - ETA: 6337 - ETA: 6336 - ETA: 6335 - ETA: 6333 - ETA: 6331 - ETA: 6332 - ETA: 6333 - ETA: 6329 - ETA: 6327 - ETA: 6328 - ETA: 6329 - ETA: 6327 - ETA: 6328 - ETA: 6326 - ETA: 6328 - ETA: 6330 - ETA: 6330 - ETA: 6330 - ETA: 6331 - ETA: 6333 - ETA: 6334 - ETA: 6334 - ETA: 6336 - ETA: 6337 - ETA: 6337 - ETA: 6344 - ETA: 6354 - ETA: 6358 - ETA: 6366 - ETA: 6372 - ETA: 6379 - ETA: 6382 - ETA: 6386 - ETA: 6389 - ETA: 6392 - ETA: 6395 - ETA: 6402 - ETA: 6418 - ETA: 6428

 87580672/553467096 [===>..........................] - ETA: 6370 - ETA: 6371 - ETA: 6372 - ETA: 6372 - ETA: 6372 - ETA: 6372 - ETA: 6373 - ETA: 6373 - ETA: 6373 - ETA: 6374 - ETA: 6374 - ETA: 6373 - ETA: 6373 - ETA: 6372 - ETA: 6371 - ETA: 6370 - ETA: 6368 - ETA: 6366 - ETA: 6365 - ETA: 6364 - ETA: 6362 - ETA: 6358 - ETA: 6358 - ETA: 6355 - ETA: 6346 - ETA: 6343 - ETA: 6340 - ETA: 6338 - ETA: 6337 - ETA: 6332 - ETA: 6331 - ETA: 6327 - ETA: 6327 - ETA: 6318 - ETA: 6317 - ETA: 6313 - ETA: 6312 - ETA: 6309 - ETA: 6308 - ETA: 6309 - ETA: 6302 - ETA: 6301 - ETA: 6298 - ETA: 6297 - ETA: 6295 - ETA: 6293 - ETA: 6289 - ETA: 6288 - ETA: 6286 - ETA: 6287 - ETA: 6281 - ETA: 6279 - ETA: 6278 - ETA: 6277 - ETA: 6276 - ETA: 6274 - ETA: 6273 - ETA: 6272 - ETA: 6270 - ETA: 6268 - ETA: 6267 - ETA: 6268 - ETA: 6264 - ETA: 6262 - ETA: 6262 - ETA: 6260 - ETA: 6259 - ETA: 6260 - ETA: 6257 - ETA: 6257 - ETA: 6256 - ETA: 6256 - ETA: 6256 - ETA: 6256 - ETA: 6255 - ETA: 6255 - ETA: 6254 - ETA: 6254 - ETA: 6255

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\xiaogao\\.keras\\models\\vgg16_weights_tf_dim_ordering_tf_kernels.h5'

**Observation**

- Although the model hasn't seen the images in the training set, it does have seen the image type "cat" from imagenet. So the model can provide fairly accurate prediction results.
- The result of preprocessing may not be always helpful. The probability of 1st image is down from 65% to 15% while that of 3rd image is up from 71% to 99%.

## Method 2: Apply pre-trained model on training data but with new classifer

**Two solutions**
- Solution 1: Load the entire pre-trained model plus weights, replace with new classifier.
- Solution 2: Create the model with the same structure of the pre-trained model and only load the pre-trained weights, replace with new classifier.

### 2.0 Data Preparation

**Steps**
- Read input：cv2.imread
- Change image size：cv2.resize
- Define image type：cat = 0, dog = 1
- Shuffle the sequence of images and classification：shuffle
- Split training set into training and validation sets：train_test_split

In [3]:
shape = 224 # VGG16 input size = 224*224
label = np.array([0] * 12500 + [1] * 12500)
data = np.zeros((25000, shape, shape, 3), dtype=np.uint8)

for i in tqdm(range(12500)):
    img = cv2.imread('./train/cat.%s.jpg' % str(i))
    img = img[:, :, ::-1]
    data[i] = cv2.resize(img, (shape, shape))
    
for i in tqdm(range(12500)):
    img = cv2.imread('./train/dog.%s.jpg' % str(i))
    img = img[:, :, ::-1]
    data[i + 12500] = cv2.resize(img, (shape, shape))
    
print('Training Data Size = %.2f GB' % (sys.getsizeof(data)/1024**3))

100%|██████████| 12500/12500 [01:54<00:00, 109.38it/s]

Training Data Size = 3.00 GB


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data, label, test_size=0.2, random_state=42)

In [6]:
test = np.zeros((12500, shape, shape, 3), dtype=np.uint8)
for i in tqdm(range(12500)):
    img = cv2.imread('./test/%s.jpg' % str(i + 1))
    img = img[:, :, ::-1]
    test[i] = cv2.resize(img, (shape, shape))
print('Testing Data Size = %.2f GB' % (sys.getsizeof(test)/1024**3))

100%|██████████| 12500/12500 [01:30<00:00, 138.42it/s]

Testing Data Size = 1.00 GB


### 2.1 Load the entire pre-trained model plus weights, replace with new classifier.

**Steps**
- Load pre-trained model and weight, excluding the original classifier
- Lock the layers of the pre-trained model to prevent the change during the training process: layers.trainable = False
- Add the new classifier to the end of the model. Choose sigmoid or softmax based on the number of types that need to classify
- Compire the model.Choose binary_crossentropy or categorical_crossentropy based on the number of types that need to classify
- Check the number of trainable weights: create function get_param_count() to count unlocked parameters
- Train the model
  - Use small batch size so that we can get high precision even with a few epochs
  - No need to use too many epochs, 5 ~ 10 is enough. Because the number of trainable parameters is small (~500), it can achieve the optimal without too many epochs.

**First I use VGG16 (22 layers) model for transfer learning.**

In [7]:
from keras import backend as K

def get_params_count(model):
    trainable = int(np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
    non_trainable = int(np.sum([K.count_params(p) for p in set(model.non_trainable_weights)]))
    return trainable, non_trainable

In [8]:
base_model = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False, pooling='avg')

for layers in base_model.layers:
    layers.trainable = False

y = Dropout(0.5)(base_model.output)
y = Dense(1, activation='sigmoid')(y)

model1 = Model(inputs=base_model.input, outputs=y)
model1.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
model1.summary()
print('Model has %d layers.' % len(model1.layers))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
model1.fit(x=X_train, y=y_train, batch_size=16, epochs=5, validation_data=(X_val, y_val), shuffle=True)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 509s - loss: 1.1711 - acc: 0.8045 - val_loss: 0.1745 - val_acc: 0.9540
Epoch 2/5
20000/20000 [==============================] - 484s - loss: 0.3822 - acc: 0.9157 - val_loss: 0.1281 - val_acc: 0.9636
Epoch 3/5
20000/20000 [==============================] - 483s - loss: 0.2848 - acc: 0.9304 - val_loss: 0.1187 - val_acc: 0.9658
Epoch 4/5
20000/20000 [==============================] - 484s - loss: 0.2421 - acc: 0.9394 - val_loss: 0.1072 - val_acc: 0.9662
Epoch 5/5
20000/20000 [==============================] - 484s - loss: 0.2120 - acc: 0.9415 - val_loss: 0.0973 - val_acc: 0.9680


**Now let's try ResNet50 model (178 layers). **

In [10]:
base_model = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False, pooling='avg')

for layers in base_model.layers:
    layers.trainable = False

y = Dropout(0.25)(base_model.output)
y = Dense(1, activation='sigmoid')(y)

model2 = Model(inputs=base_model.input, outputs=y)
model2.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
model2.summary()
print('Model has %d layers.' % len(model2.layers))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           input_2[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [11]:
model2.fit(x=X_train, y=y_train, batch_size=16, epochs=5, validation_data=(X_val, y_val), shuffle=True)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 297s - loss: 0.1764 - acc: 0.9321 - val_loss: 0.0751 - val_acc: 0.9748
Epoch 2/5
20000/20000 [==============================] - 296s - loss: 0.1040 - acc: 0.9605 - val_loss: 0.0639 - val_acc: 0.9766
Epoch 3/5
20000/20000 [==============================] - 296s - loss: 0.0979 - acc: 0.9624 - val_loss: 0.0603 - val_acc: 0.9770
Epoch 4/5
20000/20000 [==============================] - 296s - loss: 0.0926 - acc: 0.9644 - val_loss: 0.0574 - val_acc: 0.9776
Epoch 5/5
20000/20000 [==============================] - 296s - loss: 0.0868 - acc: 0.9657 - val_loss: 0.0566 - val_acc: 0.9772


### 2.2 Create the model with the same structure of the pre-trained model and only load the pre-trained weights, replace with new classifier

- It's much complicated than solution 1 as I have to create the model with the same structure by hand.
- It requires the model I create is the same as the pre-trained model (the size of all the weight matrixes should be the same). Otherwise, it will throw the error when loading the weights. The advantage of this solution is I can change the input size, hyper-paramters freely. For some dataset with small image size (e.g. CIRAR10), I have to either enlarge the image size or use solution 2 to modify the input size. 

In [12]:
x = Input(shape=(32, 32, 3))
y = x
y = Convolution2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='same')(y)

y = Convolution2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='same')(y)

y = Convolution2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='same')(y)

y = Convolution2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='same')(y)

y = Convolution2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='same')(y)

y = GlobalAveragePooling2D()(y)
# y = Dropout(0.5)(y)
# y = Dense(10, activation='softmax')(y)

modelx = Model(inputs=x, outputs=y)

In [14]:
modelx.load_weights('/home/ubuntu/.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
modelx.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 128)         0         
__________

In [15]:
modelx.layers

In [16]:
[weights, bias] = modelx.layers[1].get_weights()
print(weights.shape)
print(bias.shape)

(3, 3, 3, 64)
(64,)


In [17]:
weights[:, :, :, 0]

array([[[ 0.42947057,  0.55037946,  0.4800154 ],
        [ 0.373467  ,  0.44007453,  0.4085474 ],
        [-0.06136011, -0.08138704, -0.06514555]],

       [[ 0.27476987,  0.34573907,  0.31047726],
        [ 0.03868078,  0.04063221,  0.05020237],
        [-0.36722335, -0.45350131, -0.40338343]],

       [[-0.05746817, -0.05863491, -0.05087169],
        [-0.26224968, -0.33066967, -0.28522751],
        [-0.35009676, -0.4850302 , -0.41851634]]], dtype=float32)

In [18]:
bias[0]

0.73429835

## Method 3: Apply pre-trained model on training data with new classifer and fine tune the last several layers

**Steps**
- Here I use the model2 created above. 
- Unblock the last several layers and train them 选定模型中较靠后的部分解冻，参与训练
- Since several layers form a combination (e.g. conv + batchNorm + acitvation), we'd better block/unblock the combination.
- Unblock the layers from high (output) to low (input). Train 5 epochs when unblocking on part. Then check if the loss and accuracy have improved. If the answer is yes, then we can continue training. Otherwise, we can unblock more layers until the training result has improvement.

In [19]:
model2.layers[-37:]

In [20]:
for layers in model2.layers[-35:]:
    layers.trainable = True
    
print('Trainable = %d, Non-Trainable = %d' % (get_params_count(model2)))

Trainable = 14453249, Non-Trainable = 9136512


In [21]:
model2.fit(x=X_train, y=y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 294s - loss: 0.0860 - acc: 0.9668 - val_loss: 0.0604 - val_acc: 0.9756
Epoch 2/10
20000/20000 [==============================] - 295s - loss: 0.0870 - acc: 0.9647 - val_loss: 0.0583 - val_acc: 0.9800
Epoch 3/10
20000/20000 [==============================] - 295s - loss: 0.0795 - acc: 0.9698 - val_loss: 0.0535 - val_acc: 0.9788
Epoch 4/10
20000/20000 [==============================] - 295s - loss: 0.0830 - acc: 0.9667 - val_loss: 0.0519 - val_acc: 0.9796
Epoch 5/10
20000/20000 [==============================] - 295s - loss: 0.0841 - acc: 0.9675 - val_loss: 0.0520 - val_acc: 0.9810
Epoch 6/10
20000/20000 [==============================] - 295s - loss: 0.0796 - acc: 0.9687 - val_loss: 0.0518 - val_acc: 0.9788
Epoch 7/10
20000/20000 [==============================] - 295s - loss: 0.0799 - acc: 0.9689 - val_loss: 0.0510 - val_acc: 0.9806
Epoch 8/10
20000/20000 [========================

In [22]:
model2.save('ResNet_Finetune_last3_epoch5.h5')